## Lab 4: 프로덕션 배포 - 관찰 가능성을 갖춘 AgentCore Runtime 사용

### 개요

Lab 3에서는 보안 인증을 통해 AgentCore Gateway로 도구를 중앙화하여 고객 지원 에이전트를 확장했습니다. 이제 포괄적인 관찰 가능성을 갖춘 AgentCore Runtime에 에이전트를 배포하여 프로덕션 여정을 완료하겠습니다. 이를 통해 프로토타입을 완전한 모니터링과 자동 확장으로 실제 트래픽을 처리할 수 있는 프로덕션 준비 시스템으로 변환할 것입니다.

[Amazon Bedrock AgentCore Runtime](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/agents-tools-runtime.html)은 프레임워크, 프로토콜 또는 모델 선택에 관계없이 조직이 프로덕션에서 AI 에이전트를 배포하고 확장할 수 있도록 지원하는 안전하고 완전 관리형 런타임입니다. 엔터프라이즈급 안정성, 자동 확장 및 포괄적인 모니터링 기능을 제공합니다.

**워크샵 여정:**

- **Lab 1 (완료):** 에이전트 프로토타입 생성 - 기능적인 고객 지원 에이전트 구축
- **Lab 2 (완료):** 메모리로 향상 - 대화 컨텍스트 및 개인화 추가
- **Lab 3 (완료):** Gateway 및 Identity로 확장 - 에이전트 간 도구 안전하게 공유
- **Lab 4 (현재):** 프로덕션 배포 - 관찰 가능성을 갖춘 AgentCore Runtime 사용
- **Lab 5:** 사용자 인터페이스 구축 - 고객 대면 애플리케이션 생성

### AgentCore Runtime 및 프로덕션 배포가 중요한 이유

현재 상태 (Lab 1-3): 에이전트가 중앙화된 도구로 로컬에서 실행되지만 프로덕션 과제에 직면:

- 에이전트가 단일 세션에서 로컬로 실행
- 포괄적인 모니터링 또는 디버깅 기능 없음
- 여러 동시 사용자를 안정적으로 처리할 수 없음

이 랩 후에는 다음과 같은 프로덕션 준비 에이전트 인프라를 갖게 됩니다:

- 가변 수요를 처리하는 서버리스 자동 확장
- 추적, 메트릭 및 로깅을 통한 포괄적인 관찰 가능성
- 자동 오류 복구를 통한 엔터프라이즈 안정성
- 적절한 액세스 제어를 통한 보안 배포
- AWS 콘솔 및 API를 통한 쉬운 관리 및 실제 프로덕션 워크로드 지원


### AgentCore Observability로 포괄적인 관찰 가능성 추가

또한 AgentCore Runtime은 [AgentCore Observability](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/observability.html)와 원활하게 통합되어 프로덕션에서 에이전트의 동작에 대한 완전한 가시성을 제공합니다. AgentCore Observability는 에이전트 상호작용, 도구 사용 및 메모리 액세스 패턴에서 추적, 메트릭 및 로그를 자동으로 캡처합니다. 이 랩에서는 AgentCore Runtime이 CloudWatch GenAI Observability와 통합되어 포괄적인 모니터링 및 디버깅 기능을 제공하는 방법을 살펴보겠습니다.

요청 추적의 경우, AgentCore Observability는 도구 호출, 메모리 검색 및 모델 상호작용을 포함한 완전한 대화 흐름을 캡처합니다. 성능 모니터링의 경우, 응답 시간, 성공률 및 리소스 사용률을 추적하여 에이전트의 성능을 최적화하는 데 도움을 줍니다.

관찰 가능성 흐름 중에 AgentCore Runtime은 자동으로 에이전트 코드를 계측하고 원격 측정 데이터를 CloudWatch로 전송합니다. 그런 다음 CloudWatch 대시보드 및 GenAI Observability 기능을 사용하여 패턴을 분석하고, 병목 지점을 식별하며, 실시간으로 문제를 해결할 수 있습니다.

### Lab 4 아키텍처
<div style="text-align:left"> 
    <img src="images/architecture_lab4_runtime.png" width="75%"/> 
</div>

*에이전트는 이제 CloudWatch를 통한 완전한 관찰 가능성을 갖춘 AgentCore Runtime에서 실행되며, 자동 확장 및 포괄적인 모니터링으로 프로덕션 트래픽을 처리합니다. 이전 랩의 메모리 및 Gateway 통합은 프로덕션 환경에서 완전히 기능합니다.*

### 주요 기능

- **서버리스 에이전트 배포:** 최소한의 코드 변경으로 AgentCore Runtime을 사용하여 로컬 에이전트를 확장 가능한 프로덕션 서비스로 변환
- **포괄적인 관찰 가능성:** CloudWatch GenAI Observability를 통한 완전한 요청 추적, 성능 메트릭 및 디버깅 기능

### 전제 조건

- Python 3.12+
- 적절한 권한을 가진 AWS 계정
- Docker, Finch 또는 Podman 설치 및 실행
- Amazon Bedrock AgentCore SDK
- Strands Agents 프레임워크

**참고**: CloudWatch에서 AgentCore Observability 추적을 보려면 [CloudWatch Transaction Search](https://docs.aws.amazon.com/AmazonCloudWatch/latest/monitoring/Enable-TransactionSearch.html)를 반드시 활성화해야 합니다.


### 1단계: 필요한 라이브러리 가져오기

In [ ]:
# 필요한 라이브러리 가져오기
import os
import json
import boto3
from strands import Agent
from strands.models import BedrockModel
from lab_helpers.lab2_memory import create_or_get_memory_resource

create_or_get_memory_resource()  # 메모리 랩이 실행되지 않은 경우를 대비해서

### 2단계: AgentCore Runtime용 에이전트 준비

#### 런타임 준비 에이전트 생성

먼저 이전 로컬 에이전트 구현 내에서 Python SDK를 통해 필요한 AgentCore Runtime 구성 요소를 정의해보겠습니다.

아래의 `#### AGENTCORE RUNTIME - LINE i ####` 주석을 관찰하여 관련 배포 코드가 추가된 위치를 확인하세요. 런타임 준비 에이전트를 준비하는 4개의 라인을 찾을 수 있습니다:

1. `from bedrock_agentcore.runtime import BedrockAgentCoreApp`로 Runtime App 가져오기
2. `app = BedrockAgentCoreApp()`로 App 초기화
3. `@app.entrypoint`로 호출 함수 데코레이트
4. `app.run()`으로 AgentCore Runtime이 실행을 제어하도록 함


In [ ]:
%%writefile ./lab_helpers/lab4_runtime.py
from bedrock_agentcore.runtime import (
    BedrockAgentCoreApp,
)  #### AGENTCORE RUNTIME - LINE 1 ####
from strands import Agent
from strands.models import BedrockModel
from scripts.utils import get_ssm_parameter
from lab_helpers.lab1_strands_agent import (
    get_return_policy,
    get_product_info,
    SYSTEM_PROMPT,
    MODEL_ID,
)

from lab_helpers.lab2_memory import (
    CustomerSupportMemoryHooks,
    memory_client,
    ACTOR_ID,
    SESSION_ID,
)

# Lab1 가져오기: Bedrock 모델 생성
model = BedrockModel(model_id=MODEL_ID)

# Lab2 가져오기: 훅을 통한 메모리 초기화
memory_id = get_ssm_parameter("/app/customersupport/agentcore/memory_id")
memory_hooks = CustomerSupportMemoryHooks(
    memory_id, memory_client, ACTOR_ID, SESSION_ID
)

# 모든 고객 지원 도구를 포함한 에이전트 생성
agent = Agent(
    model=model,
    tools=[get_return_policy, get_product_info],
    system_prompt=SYSTEM_PROMPT,
    hooks=[memory_hooks],
)

# AgentCore Runtime 앱 초기화
app = BedrockAgentCoreApp()  #### AGENTCORE RUNTIME - LINE 2 ####


@app.entrypoint  #### AGENTCORE RUNTIME - LINE 3 ####
def invoke(payload):
    """AgentCore Runtime 진입점 함수"""
    user_input = payload.get("prompt", "")

    # 에이전트 호출
    response = agent(user_input)
    return response.message["content"][0]["text"]


if __name__ == "__main__":
    app.run()  #### AGENTCORE RUNTIME - LINE 4 ####

#### 내부적으로 무엇이 일어나는가?

`BedrockAgentCoreApp`을 사용하면 자동으로:

- 포트 8080에서 수신하는 HTTP 서버 생성
- 요청 처리를 위한 필수 `/invocations` 엔드포인트 구현
- 상태 확인을 위한 `/ping` 엔드포인트 구현
- 적절한 콘텐츠 유형 및 응답 형식 처리
- AWS 표준에 따른 오류 처리 관리


### 3단계: AgentCore Runtime에 배포


`CreateAgentRuntime` 작업은 포괄적인 구성 옵션을 지원하여 컨테이너 이미지, 환경 변수 및 암호화 설정을 지정할 수 있습니다. 또한 프로토콜 설정(HTTP, MCP) 및 권한 부여 메커니즘을 구성하여 클라이언트가 에이전트와 통신하는 방법을 제어할 수 있습니다. 

**참고:** 운영 모범 사례는 코드를 컨테이너로 패키징하고 CI/CD 파이프라인과 IaC를 사용하여 ECR에 푸시하는 것입니다

이 튜토리얼에서는 Amazon Bedrock AgentCore Python SDK를 사용하여 아티팩트를 쉽게 패키징하고 AgentCore 런타임에 배포합니다.

이제 [AgentCore Starter Toolkit](https://github.com/aws/bedrock-agentcore-starter-toolkit)을 사용하여 에이전트를 AgentCore Runtime에 배포해보겠습니다.

#### 보안 런타임 배포 구성 (AgentCore Runtime + AgentCore Identity)

먼저 스타터 툴킷을 사용하여 진입점, 생성할 실행 역할 및 요구사항 파일로 AgentCore Runtime 배포를 구성합니다. 또한 Amazon Cognito 사용자 풀을 사용하여 ID 인증을 구성하고 시작 시 Amazon ECR 리포지토리를 자동 생성하도록 스타터 킷을 구성합니다.

구성 단계에서 애플리케이션 코드를 기반으로 도커 파일이 생성됩니다

<div style="text-align:left"> 
    <img src="images/configure.png" width="75%"/> 
</div>

**참고**: Cognito access_token은 2시간 동안만 유효합니다. access_token이 만료되면 `reauthenticate_user` 메서드를 사용하여 다른 access_token을 발급받을 수 있습니다.


In [ ]:
from lab_helpers.utils import setup_cognito_user_pool, reauthenticate_user

print("Setting up Amazon Cognito user pool...")
cognito_config = (
    setup_cognito_user_pool()
)  # You'll get your bearer token from this output cell.
print("Cognito setup completed ✓")

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from lab_helpers.utils import create_agentcore_runtime_execution_role

# Initialize the runtime toolkit
boto_session = boto3.session.Session()
region = boto_session.region_name

execution_role_arn = create_agentcore_runtime_execution_role()

agentcore_runtime = Runtime()

# Configure the deployment
response = agentcore_runtime.configure(
    entrypoint="lab_helpers/lab4_runtime.py",
    execution_role=execution_role_arn,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name="customer_support_agent",
    authorizer_configuration={
        "customJWTAuthorizer": {
            "allowedClients": [cognito_config.get("client_id")],
            "discoveryUrl": cognito_config.get("discovery_url"),
        }
    },
)

print("Configuration completed:", response)

### AgentCore Runtime에 에이전트 시작

이제 도커 파일이 있으므로 에이전트를 AgentCore Runtime에 시작해보겠습니다. 이렇게 하면 Amazon ECR 리포지토리와 AgentCore Runtime이 생성됩니다

<div style="text-align:left"> 
    <img src="images/launch.png" width="100%"/> 
</div>

In [ ]:
# Launch the agent (this will build and deploy the container)
from lab_helpers.utils import put_ssm_parameter

launch_result = agentcore_runtime.launch()
print("Launch completed:", launch_result.agent_arn)

agent_arn = put_ssm_parameter(
    "/app/customersupport/agentcore/runtime_arn", launch_result.agent_arn
)

### AgentCore Runtime 상태 확인
이제 AgentCore Runtime을 배포했으므로 배포 상태를 확인해보겠습니다

In [ ]:
import time

# Wait for the agent to be ready
status_response = agentcore_runtime.status()
status = status_response.endpoint["status"]

end_status = ["READY", "CREATE_FAILED", "DELETE_FAILED", "UPDATE_FAILED"]
while status not in end_status:
    print(f"Waiting for deployment... Current status: {status}")
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint["status"]

print(f"Final status: {status}")

### AgentCore Runtime 호출

마지막으로 페이로드로 AgentCore Runtime을 호출할 수 있습니다

이제 에이전트가 배포되고 준비되었으므로 몇 가지 쿼리로 테스트해보겠습니다. 올바른 인증 토큰 유형으로 에이전트를 호출합니다. 우리의 경우 Cognito 액세스 토큰이 될 것입니다. 위 셀에서 액세스 토큰을 복사하세요

<div style="text-align:left"> 
    <img src="images/invoke.png" width="100%"/> 
</div>

#### AgentCore Starter Toolkit 사용

AgentCore Starter Toolkit을 사용하여 호출을 통해 에이전트가 작동하는지 검증할 수 있습니다. 스타터 툴킷은 에이전트를 쿼리하기 위해 자동으로 세션 ID를 생성할 수 있습니다. 또는 호출 중에 세션 ID를 매개변수로 전달할 수도 있습니다. 시연 목적으로 자체 세션 ID를 생성하겠습니다.

In [ ]:
import uuid

# Create a session ID for demonstrating session continuity
session_id = uuid.uuid4()

# Test different customer support scenarios
user_query = "제 갤럭시가 블루투스에 연결되지 않아요. 어떻게 해야 하나요?"

bearer_token = reauthenticate_user(
    cognito_config.get("client_id"), 
    cognito_config.get("client_secret")
)

response = agentcore_runtime.invoke(
    {"prompt": user_query}, 
    bearer_token=bearer_token,
    session_id=str(session_id)
)
response

#### 세션 연속성으로 에이전트 호출

AgentCore Runtime을 사용하고 있으므로 동일한 세션 ID로 대화를 쉽게 계속할 수 있습니다.

In [ ]:
user_query = "블루투스를 켜고 껐지만 여전히 작동하지 않습니다."
response = agentcore_runtime.invoke(
    {"prompt": user_query}, 
    bearer_token=bearer_token,
    session_id=str(session_id)
)
response

#### 새 사용자로 에이전트 호출
아래 예제에서는 두 번째 쿼리에서 iPhone 기기를 언급하지 않았지만 에이전트는 여전히 그 컨텍스트를 가지고 있습니다. 이는 AgentCore Runtime 세션 연속성 때문입니다. 에이전트는 새 사용자의 컨텍스트를 알지 못합니다.

In [ ]:
# Creating a new session ID for demonstrating new customer
session_id2 = uuid.uuid4()

user_query = "아직도 작동하지 않아요. 왜그러죠?"
response = agentcore_runtime.invoke(
    {"prompt": user_query}, 
    bearer_token=bearer_token,
    session_id=str(session_id2)
)
response

이 경우 에이전트는 더 이상 컨텍스트를 가지지 않으며 더 많은 정보가 필요합니다.

그리고 이것이 모든 기본 인프라를 관리할 필요 없이 에이전트를 위한 안전하고 확장 가능한 엔드포인트를 갖는 데 필요한 전부입니다!

### 5단계: AgentCore Observability

[AgentCore Observability](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/observability.html)는 Amazon OpenTelemetry Python Instrumentation 및 Amazon CloudWatch GenAI Observability를 사용하여 AI 에이전트에 대한 모니터링 및 추적 기능을 제공합니다.

#### 에이전트

기본 AgentCore Runtime 구성은 **AgentCore Observability**를 통해 CloudWatch에서 에이전트의 추적을 로깅할 수 있게 합니다. 이러한 추적은 AWS CloudWatch GenAI Observability 대시보드에서 볼 수 있습니다. Cloudwatch → GenAI Observability → Bedrock AgentCore로 이동하세요.

![CloudWatch의 에이전트 개요](images/observability_agents.png)

#### 세션

세션 보기는 계정의 모든 에이전트와 연결된 모든 세션의 목록을 보여줍니다.

![세션](images/sessions_lab5_observability.png)

#### 추적

추적 보기는 이 계정의 에이전트에서 모든 추적을 나열합니다. 추적 작업을 위해:

- 특정 추적을 검색하려면 Filter traces를 선택하세요.
- 결과를 정리하려면 열 이름으로 정렬하세요.
- Actions 아래에서 로그 및 스팬 데이터를 쿼리하여 검색을 세분화하려면 Logs Insights를 선택하거나 내보내려면 Export selected traces를 선택하세요.

![추적](images/traces_lab4_observability.png)


### 축하합니다! 🎉

**Lab 4: 프로덕션 배포 - 관찰 가능성을 갖춘 AgentCore Runtime 사용!**을 성공적으로 완료했습니다!

달성한 내용은 다음과 같습니다:

##### 프로덕션 준비 배포:

- 최소한의 코드 변경(4줄만 추가)으로 프로덕션용 에이전트 준비
- 서로 다른 고객 간의 적절한 세션 격리 검증
- 세션별 세션 연속성 + 메모리 지속성 및 컨텍스트 인식 확인

##### 엔터프라이즈급 보안 및 ID:

- JWT 토큰과 Cognito 통합을 사용한 보안 인증 구현
- 프로덕션 워크로드를 위한 적절한 IAM 역할 및 실행 권한 구성
- 보안 에이전트 호출을 위한 ID 기반 액세스 제어 설정

##### 포괄적인 관찰 가능성:

- 모든 고객 세션에 걸친 완전한 요청 추적을 위한 AgentCore Observability 활성화
- CloudWatch GenAI Observability 대시보드 모니터링 구성

##### 현재 제한사항 (다음에 해결할 예정!):

- **개발자 중심 상호작용** - SDK/API 호출을 통해 에이전트에 액세스할 수 있지만 사용자 친화적인 웹 인터페이스 없음
- **수동 세션 관리** - 직관적인 사용자 경험보다는 프로그래밍 방식의 세션 생성 필요

##### 다음 단계 [Lab 5: 사용자 인터페이스 구축 →](lab-05-frontend.ipynb)
Lab 5에서는 사용자 친화적인 인터페이스를 구축하여 고객 경험을 완성할 것입니다!! 가자!!
